# Attention mechanisms and transformers

One major limitation of recurrent networks is that all words in a sequence are treated as having the same influence on the result. This leads to suboptimal performance in standard LSTM encoder-decoder models for sequence-to-sequence tasks, such as Named Entity Recognition and Machine Translation. In reality, certain words in the input sequence often have a greater impact on the sequential outputs than others.

Consider a sequence-to-sequence model, such as machine translation. It is implemented using two recurrent networks: one network (**encoder**) compresses the input sequence into a hidden state, and another network (**decoder**) expands this hidden state into the translated output. The issue with this approach is that the final state of the network struggles to retain information from the beginning of a sentence, resulting in poor model performance on longer sentences.

**Attention Mechanisms** offer a way to assign different levels of importance to each input vector when predicting each output of the RNN. This is achieved by creating shortcuts between the intermediate states of the input RNN and the output RNN. In this way, when generating the output symbol $y_t$, all input hidden states $h_i$ are considered, but with varying weight coefficients $\alpha_{t,i}$.

![Image showing an encoder/decoder model with an additive attention layer](../../../../../translated_images/en/encoder-decoder-attention.7a726296894fb567aa2898c94b17b3289087f6705c11907df8301df9e5eeb3de.png)
*The encoder-decoder model with additive attention mechanism in [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), cited from [this blog post](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

The attention matrix $\{\alpha_{i,j}\}$ represents the extent to which specific input words contribute to the generation of a particular word in the output sequence. Below is an example of such a matrix:

![Image showing a sample alignment found by RNNsearch-50, taken from Bahdanau - arviz.org](../../../../../translated_images/en/bahdanau-fig3.09ba2d37f202a6af11de6c82d2d197830ba5f4528d9ea430eb65fd3a75065973.png)

*Figure taken from [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Attention mechanisms are responsible for much of the current or near-current state-of-the-art advancements in Natural Language Processing. However, adding attention significantly increases the number of model parameters, which led to scaling challenges with RNNs. A key limitation of scaling RNNs is that their sequential nature makes it difficult to batch and parallelize training. In an RNN, each element of a sequence must be processed in order, which prevents easy parallelization.

The adoption of attention mechanisms, combined with this limitation, led to the development of the now state-of-the-art Transformer Models that we use today, from BERT to OpenGPT3.

## Transformer models

Instead of passing the context of each previous prediction into the next evaluation step, **transformer models** use **positional encodings** and **attention** to capture the context of a given input within a specified window of text. The image below illustrates how positional encodings combined with attention can capture context within a given window.

![Animated GIF showing how the evaluations are performed in transformer models.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Since each input position is mapped independently to each output position, transformers can parallelize much better than RNNs, enabling the creation of larger and more expressive language models. Each attention head can learn different relationships between words, enhancing performance in downstream Natural Language Processing tasks.

## Building Simple Transformer Model

Keras does not include a built-in Transformer layer, but we can construct one ourselves. As before, we will focus on text classification using the AG News dataset. However, it is worth noting that Transformer models achieve their best results on more complex NLP tasks.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

New layers in Keras should subclass `Layer` class, and implement `call` method. Let's start with **Positional Embedding** layer. We will use [some code from official Keras documentation](https://keras.io/examples/nlp/text_classification_with_transformer/). We will assume that we pad all input sequences to length `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Now, let's implement the transformer block. It will use the output from the previously defined embedding layer:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Now, we can proceed to define the complete transformer model:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Models

**BERT** (Bidirectional Encoder Representations from Transformers) is a very large multi-layer transformer network with 12 layers for *BERT-base* and 24 for *BERT-large*. The model is initially pre-trained on a large corpus of text data (Wikipedia + books) using unsupervised training (predicting masked words in a sentence). During pre-training, the model gains a significant level of language understanding, which can then be applied to other datasets through fine-tuning. This process is known as **transfer learning**.

![picture from http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/en/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362e39ee4381aab7cad06b5465a0b5f053a0f2aa05fbe14e746.png)

There are many variations of Transformer architectures, including BERT, DistilBERT, BigBird, OpenGPT3, and more, which can be fine-tuned.

Let's explore how we can use a pre-trained BERT model to solve a traditional sequence classification problem. We will take inspiration and some code from the [official documentation](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

To load pre-trained models, we will use **TensorFlow Hub**. First, let's load the BERT-specific vectorizer:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

It is important to use the same vectorizer that was used to train the original network. Additionally, the BERT vectorizer provides three components:
* `input_word_ids`, which represents a sequence of token IDs for the input sentence
* `input_mask`, indicating which parts of the sequence contain actual input and which parts are padding. This is similar to the mask generated by the `Masking` layer
* `input_type_ids`, used for language modeling tasks, allowing you to specify two input sentences within a single sequence.

Next, we can initialize the BERT feature extractor:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


So, the BERT layer provides several useful outputs:
* `pooled_output` is the result of averaging all tokens in the sequence. You can think of it as a smart semantic embedding of the entire network. It is similar to the output of the `GlobalAveragePooling1D` layer in our previous model.
* `sequence_output` is the output of the last transformer layer (corresponds to the output of `TransformerBlock` in the model above).
* `encoder_outputs` are the outputs of all transformer layers. Since we loaded a 4-layer BERT model (as you might infer from the name, which includes `4_H`), it contains 4 tensors. The last one is identical to `sequence_output`.

Next, we will define the end-to-end classification model. We'll use a *functional model definition*, where we specify the model input and then provide a series of expressions to compute its output. Additionally, we'll set the BERT model weights to be non-trainable and only train the final classifier:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Despite having few trainable parameters, the process is quite slow because the BERT feature extractor is computationally intensive. It seems we couldn't achieve satisfactory accuracy, possibly due to insufficient training or a lack of model parameters.

Let's attempt to unfreeze the BERT weights and train them as well. This will require a very small learning rate and a more cautious training strategy, including **warmup**, and using the **AdamW** optimizer. We'll utilize the `tf-models-official` package to set up the optimizer:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

As you can see, training progresses quite slowly—but you might want to experiment by training the model for a few epochs (5-10) to see if you can achieve better results compared to the methods we've used previously.

## Huggingface Transformers Library

Another very common (and slightly simpler) way to work with Transformer models is through the [HuggingFace package](https://github.com/huggingface/), which offers straightforward building blocks for various NLP tasks. It supports both TensorFlow and PyTorch, another widely-used neural network framework.

> **Note**: If you're not interested in exploring how the Transformers library works, you can skip to the end of this notebook. You won't encounter anything significantly different from what we've already done. We'll essentially repeat the same steps to train a BERT model using a different library and a much larger model. This process involves fairly lengthy training, so you might prefer to simply review the code. 

Now, let's explore how we can solve our problem using [Huggingface Transformers](http://huggingface.co).


The first thing we need to do is choose the model we will use. In addition to some built-in models, Huggingface offers an [online model repository](https://huggingface.co/models), where you can find many more pre-trained models created by the community. All of these models can be loaded and used simply by providing a model name. The necessary binary files for the model will be downloaded automatically.

Sometimes, you may need to load your own models. In such cases, you can specify the directory that contains all the relevant files, including the tokenizer parameters, the `config.json` file with the model parameters, binary weights, and so on.

Using the model name, we can initialize both the model and the tokenizer. Let's start with the tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

The `tokenizer` object contains the `encode` function that can be directly used to encode text:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

We can also use tokenizer to encode a sequence in a way suitable for passing to the model, i.e. including `token_ids`, `input_mask` fields, etc. We can also specify that we want Tensorflow tensors by providing `return_tensors='tf'` argument:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

In our case, we will be using a pre-trained BERT model called `bert-base-uncased`. *Uncased* means that the model is not case-sensitive.

When training the model, we need to provide a tokenized sequence as input, so we will design a data processing pipeline. Since `tokenizer.encode` is a Python function, we will use the same approach as in the previous unit by calling it with `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Now we can load the actual model using `BertForSequenceClassification` package. This ensures that our model already has a required architecture for classification, including final classifier. You will see warning message stating that weights of the final classifier are not initialized, and model would require pre-training - that is perfectly okay, because it is exactly what we are about to do!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


As you can see from `summary()`, the model contains almost 110 million parameters! Presumably, if we want a simple classification task on a relatively small dataset, we do not want to train the BERT base layer:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Now we are ready to begin training!

> **Note**: Training a full-scale BERT model can take a significant amount of time! Therefore, we will only train it for the first 32 batches. This is just to demonstrate how model training is configured. If you'd like to attempt full-scale training, simply remove the `steps_per_epoch` and `validation_steps` parameters, and be prepared to wait!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


If you increase the number of iterations, wait sufficiently, and train for several epochs, you can expect BERT classification to deliver the best accuracy! This is because BERT already has a strong understanding of language structure, and we only need to fine-tune the final classifier. However, since BERT is a large model, the entire training process is time-consuming and demands significant computational resources! (GPU, and ideally more than one).

> **Note:** In our example, we have been using one of the smallest pre-trained BERT models. Larger models are available and are likely to produce better results.


## Takeaway

In this unit, we explored the latest model architectures based on **transformers**. We applied them to our text classification task, but BERT models can also be used for tasks like entity extraction, question answering, and other NLP applications.

Transformer models represent the current state-of-the-art in NLP, and in most cases, they should be your first choice when experimenting with custom NLP solutions. However, understanding the fundamental principles of recurrent neural networks discussed in this module is crucial if you aim to develop advanced neural models.



---

**Disclaimer**:  
This document has been translated using the AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). While we strive for accuracy, please note that automated translations may contain errors or inaccuracies. The original document in its native language should be regarded as the authoritative source. For critical information, professional human translation is recommended. We are not responsible for any misunderstandings or misinterpretations resulting from the use of this translation.
